# Report Link
https://docs.google.com/document/d/1OzHshf46L-8EXjkod2YAW2c4bVLP6Dbp0NyYaKk64io/edit?usp=sharing


# Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report,accuracy_score,ConfusionMatrixDisplay
from google.colab import drive
import gc
drive.mount('/content/drive')
import numpy as np
import pandas as pd
from numpy.core.fromnumeric import size
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix 
import PIL

MessageError: ignored

# Generation of Data Matrix and Splitting Data

In [ ]:

import numpy as np
import PIL
an_image = PIL.Image.open("/content/drive/MyDrive/prassignment1/s1/1.pgm")
image_sequence = an_image.getdata() 
image_array = np.array(image_sequence)
data_matrix = np.full([10305,1],None)
for i in range(40):
  num_of_file=str(i+1)
  path="/content/drive/MyDrive/prassignment1/s{}".format(num_of_file)
  for j in range(10):
    num_of_image=str(j+1)
    full_path=path+"/"+num_of_image+".pgm"
    an_image = PIL.Image.open(full_path)
    image_sequence = an_image.getdata()
    image_array = np.array(image_sequence)
    image_array_labeled = np.insert(image_array,0,i,axis=0)
    data_matrix=np.c_[data_matrix,image_array_labeled]
    

data_matrix=data_matrix[:,1:]
data_matrix=np.transpose(data_matrix)
even_data_matrix=np.full([1,10305],None)
odd_data_matrix=np.full([1,10305],None)
for i in range(400):
  if i%2==0:
    even_data_matrix=np.append(even_data_matrix,[data_matrix[i]],axis=0)
  else:
    odd_data_matrix=np.append(odd_data_matrix,[data_matrix[i]],axis=0)
even_data_matrix=np.delete(even_data_matrix,0,0)
odd_data_matrix=np.delete(odd_data_matrix,0,0)
thirty_data_matrix=np.full([1,10305],None)
seventy_data_matrix=np.full([1,10305],None)
for i in range(40):
  for j in range(10):
    if j>6:
      thirty_data_matrix=np.append(thirty_data_matrix,[data_matrix[i*10+j]],axis=0)
    else:
      seventy_data_matrix=np.append(seventy_data_matrix,[data_matrix[i*10+j]],axis=0)
thirty_data_matrix=np.delete(thirty_data_matrix,0,0)
seventy_data_matrix=np.delete(seventy_data_matrix,0,0)

#PCA

## Data formatting

In [ ]:
dataset = odd_data_matrix[:,1:10305]
print(dataset.shape)
alpha_values = [0.8,0.85,0.9,0.95]



print(dataset)


# center data
print(dataset.mean(axis = 0).shape)
centered_ds = dataset - dataset.mean(axis = 0)
print(centered_ds.shape)


# get covariance matrix
cov_matrix = np.cov(centered_ds.astype(float).T,ddof=0)
print(cov_matrix.shape)




## Computing eigen-values & eigen-vectors


In [ ]:
# compute eigen values and eigen vectors
eig_values , eig_vectors = np.linalg.eigh(cov_matrix)
sorted_indices = eig_values.argsort()[::-1] 
eig_values = eig_values[sorted_indices]
eig_vectors = eig_vectors[:,sorted_indices]
print(eig_values)
print(eig_vectors)




In [ ]:
print(eig_values.shape)
print(eig_vectors.shape)

## Computing number of reduced dimensions


In [ ]:
# number of reduced dimensions
reduced_dimensions = []
sum_eigen_values = eig_values.sum()
for alpha in alpha_values:
  current_sum = 0.0
  i = 0
  while (current_sum/sum_eigen_values)<alpha:
      current_sum = current_sum + eig_values[i]
      i+=1
  reduced_dimensions.append(i)


# get U-matrix
print(reduced_dimensions)
#U_matrix = eig_vectors[:,0:reduced_dimensions[3]]
#U_matrices = np.array([U_matrix])
#U_matrices = np.append(U_matrices,[U_matrix],axis=0)
#print(U_matrices)

## Projecting Dataset


In [ ]:
U_matrix = eig_vectors[:,0:reduced_dimensions[3]]
print(U_matrix.shape)
projected_training_data = np.dot(centered_ds,U_matrix)
train_labels = odd_data_matrix[:,0]
print(projected_training_data.shape)
centered_testing_data = even_data_matrix[:,1:10305]
centered_testing_data = centered_testing_data - centered_testing_data.mean(axis = 0)
print(centered_testing_data.shape)
projected_testing_data = np.dot(centered_testing_data,U_matrix)
test_labels = even_data_matrix[:,0]
print(test_labels.shape)
print(projected_testing_data.shape)

## Training and testing


In [ ]:
k_nearest_Neighbours =KNeighborsClassifier(n_neighbors=1)
train_labels=train_labels.astype('int')
result_k_nearest_Neighbours = k_nearest_Neighbours.fit(projected_training_data,train_labels)
predict_for_target_k_nearest = result_k_nearest_Neighbours.predict(projected_testing_data)

In [ ]:
test_labels=test_labels.astype('int')
classification_report_result_k_nearest = classification_report(test_labels,predict_for_target_k_nearest)
print(classification_report_result_k_nearest)

## PCA function


In [ ]:
def pca(training_set,testing_set,alpha_values,k):
  dataset = training_set[:,1:10305]
  centered_ds = dataset - dataset.mean(axis = 0)
  cov_matrix = np.cov(centered_ds.astype(float).T,ddof=0)
  eig_values , eig_vectors = np.linalg.eigh(cov_matrix)
  sorted_indices = eig_values.argsort()[::-1] 
  eig_values = eig_values[sorted_indices]
  eig_vectors = eig_vectors[:,sorted_indices]
  reduced_dimensions = []
  sum_eigen_values = eig_values.sum()
  for alpha in alpha_values:
    current_sum = 0.0
    i = 0
    while (current_sum/sum_eigen_values)<alpha:
      current_sum = current_sum + eig_values[i]
      i+=1
    reduced_dimensions.append(i)
  
  current_sum = 0.0
  accuracies = list()
  classification_reports = list()
  predicted_labels = list()
  confusion_matrices = list()
  for i in reduced_dimensions:
    U_matrix = eig_vectors[:,0:i]
    projected_training_data = np.dot(centered_ds,U_matrix)
    train_labels = training_set[:,0]
    centered_testing_data = testing_set[:,1:10305]
    centered_testing_data = centered_testing_data - centered_testing_data.mean(axis = 0)
    projected_testing_data = np.dot(centered_testing_data,U_matrix)
    test_labels = testing_set[:,0]
    k_nearest_Neighbours =KNeighborsClassifier(n_neighbors=k)
    train_labels=train_labels.astype('int')
    result_k_nearest_Neighbours = k_nearest_Neighbours.fit(projected_training_data,train_labels)
    predict_for_target_k_nearest = result_k_nearest_Neighbours.predict(projected_testing_data)
    test_labels=test_labels.astype('int')
    classification_report_result_k_nearest = classification_report(test_labels,predict_for_target_k_nearest,digits=3)
    confusion_matrix_result = confusion_matrix(test_labels,predict_for_target_k_nearest)
    accuaracy_result_KNN = accuracy_score(test_labels, predict_for_target_k_nearest)
    accuracies.append(accuaracy_result_KNN)
    classification_reports.append(classification_report_result_k_nearest)
    predicted_labels.append(predict_for_target_k_nearest)
    confusion_matrices.append(confusion_matrix_result)
    if(len(alpha_values)==1):
      return accuaracy_result_KNN, classification_report_result_k_nearest , predict_for_target_k_nearest,confusion_matrix_result
  return accuracies, classification_reports , predicted_labels,confusion_matrices

In [ ]:
def compute_U_matrix(training_set,testing_set,alpha_value):
  dataset = training_set[:,1:10305]
  centered_ds = dataset - dataset.mean(axis = 0)
  cov_matrix = np.cov(centered_ds.astype(float).T,ddof=0)
  eig_values , eig_vectors = np.linalg.eigh(cov_matrix)
  sorted_indices = eig_values.argsort()[::-1] 
  eig_values = eig_values[sorted_indices]
  eig_vectors = eig_vectors[:,sorted_indices]
  #reduced_dimensions = []
  sum_eigen_values = eig_values.sum()
  current_sum = 0.0
  i = 0
  while (current_sum/sum_eigen_values)<alpha_value:
    current_sum = current_sum + eig_values[i]
    i+=1
    #reduced_dimensions.append(i)
  U_matrix = eig_vectors[:,0:i]
  return U_matrix

In [ ]:
def project_datasets(U_matrix,training_set,testing_set):
  dataset = training_set[:,1:10305]
  centered_ds = dataset - dataset.mean(axis = 0)
  centered_testing_data = testing_set[:,1:10305]
  centered_testing_data = centered_testing_data - centered_testing_data.mean(axis = 0)
  projected_training_data = np.dot(centered_ds,U_matrix)
  projected_testing_data = np.dot(centered_testing_data,U_matrix)
  
  return projected_training_data,projected_testing_data


In [ ]:
def pca_at_K(training_set,testing_set,alpha_value,k):

  U_matrix = compute_U_matrix(training_set,testing_set,alpha_value)
  
  projected_training_data,projected_testing_data = project_datasets(U_matrix,training_set,testing_set)

  train_labels = training_set[:,0]
  test_labels = testing_set[:,0]
  train_labels=train_labels.astype('int')
  test_labels=test_labels.astype('int')
  accuracies = list()

  for i in k:
    k_nearest_Neighbours =KNeighborsClassifier(n_neighbors=i)
    result_k_nearest_Neighbours = k_nearest_Neighbours.fit(projected_training_data,train_labels)
    predict_for_target_k_nearest = result_k_nearest_Neighbours.predict(projected_testing_data)
    classification_report_result_k_nearest = classification_report(test_labels,predict_for_target_k_nearest,digits=3)
    accuaracy_result_KNN = accuracy_score(test_labels, predict_for_target_k_nearest)
    accuracies.append(accuaracy_result_KNN)
  return accuracies

#LDA

## Calculate Class Means

In [ ]:
def compute_class_means(number_of_classes,number_of_features,training_data):
  class_means=np.empty((number_of_classes,number_of_features))

  for i in range(number_of_classes): 
    class_means[i]=training_data[training_data.iloc[:,0]==i].iloc[:,1:].mean(axis='index').to_numpy()
  
  return class_means


## Calculate Overall Sample Mean

In [ ]:
def compute_overall_sample_mean(training_data):
  overall_sample_mean=training_data.iloc[:,1:].mean(axis='index').to_numpy()
  return overall_sample_mean

## Calculate Between-Class Scatter Matrix

In [ ]:
def compute_between_class_scatter(number_of_classes,number_of_features,number_of_class_samples,class_means,overall_sample_mean):
 

  between_class_scatter=np.zeros((number_of_features,number_of_features))

  for i in range (number_of_classes):
    np.add(between_class_scatter,np.multiply((np.outer(class_means[i]-overall_sample_mean,class_means[i]-overall_sample_mean)),number_of_class_samples[i]),out=between_class_scatter)
  
  return between_class_scatter


## Calculate Within-Class Scatter Matrix

In [ ]:
def compute_within_class_scatter(number_of_classes,number_of_features,training_data,class_means):
    within_class_scatter=np.zeros((number_of_features,number_of_features))
    for i in range(number_of_classes):
      centralized_class_data=(training_data[training_data.iloc[:,0]==i].iloc[:,1:]-class_means[i]).to_numpy() 
      np.add(within_class_scatter,np.matmul(np.transpose(centralized_class_data),centralized_class_data),out=within_class_scatter,casting='unsafe')
      gc.collect()
    return within_class_scatter

## Compute Eigenvalues and Eigenvectors

In [ ]:
def compute_eigens(number_of_classes,number_of_features,number_of_class_samples,training_data):
  class_means=compute_class_means(number_of_classes,number_of_features,training_data)

  overall_sample_mean=compute_overall_sample_mean(training_data)

  between_class_scatter=compute_between_class_scatter(number_of_classes,number_of_features,number_of_class_samples,class_means,overall_sample_mean)
  
  within_class_scatter=compute_within_class_scatter(number_of_classes,number_of_features,training_data,class_means)

  lda_eigen_values,lda_eigen_vectors=np.linalg.eigh(np.matmul(np.linalg.inv(within_class_scatter),between_class_scatter))
  lda_eigen_values=lda_eigen_values.real
  lda_eigen_vectors=lda_eigen_vectors.real
  sorted_indices = lda_eigen_values.argsort()[::-1] 
  lda_eigen_values = lda_eigen_values[sorted_indices]
  lda_eigen_vectors = lda_eigen_vectors[:,sorted_indices]

  return lda_eigen_values,lda_eigen_vectors


## Compute Projection Matrix

In [ ]:
def compute_projection_matrix(eigen_values,eigen_vectors,number_of_features=10304,number_of_dominant_eigen_vectors=39):
  


  lda_projection_matrix=np.empty((number_of_features,number_of_dominant_eigen_vectors))

  for i in range(number_of_dominant_eigen_vectors):
    lda_projection_matrix[:,i]=eigen_vectors[:,i].real
  lda_projection_matrix=lda_projection_matrix.T

  return lda_projection_matrix



## Projection on The Training Set

In [ ]:
def project_training_set(lda_projection_matrix,training_data):
  lda_projected_training_set=np.matmul(lda_projection_matrix,training_data.iloc[:,1:].to_numpy().T).T
  return lda_projected_training_set

## Projection on The Test Set

In [ ]:
def project_testing_set(lda_projection_matrix,testing_data):
  lda_projected_testing_set=np.matmul(lda_projection_matrix,testing_data.iloc[:,1:].to_numpy().T).T
  return lda_projected_testing_set

## Project Training and Testing Data

In [ ]:
def project_data(lda_projection_matrix,training_data,testing_data):
  lda_projected_training_set=project_training_set(lda_projection_matrix,training_data)
  lda_projected_testing_set=project_testing_set(lda_projection_matrix,testing_data)
  return lda_projected_training_set,lda_projected_testing_set

## LDA Function

In [ ]:
def LDA(training_data,testing_data,number_of_class_samples,number_of_classes=40,number_of_features=10304,number_of_dominant_eigen_vectors=39):
  
 

  lda_eigen_values,lda_eigen_vectors=compute_eigens(number_of_classes,number_of_features,number_of_class_samples,training_data)

  lda_projection_matrix=compute_projection_matrix(lda_eigen_values,lda_eigen_vectors,number_of_features,number_of_dominant_eigen_vectors)

  lda_projected_training_set,lda_projected_testing_set=project_data(lda_projection_matrix,training_data,testing_data)

 
  return lda_projected_training_set,lda_projected_testing_set

In [ ]:
def KNN(n_neighbors,training_labels,testing_labels,lda_projected_training_set,lda_projected_testing_set):
  k_nearest_Neighbours =KNeighborsClassifier(n_neighbors=n_neighbors)
  result_k_nearest_Neighbours = k_nearest_Neighbours.fit(lda_projected_training_set,training_labels)
  predict_for_target_k_nearest = result_k_nearest_Neighbours.predict(lda_projected_testing_set)
  classification_report_result_k_nearest = classification_report(testing_labels,predict_for_target_k_nearest)
  accuracy = accuracy_score(testing_labels,predict_for_target_k_nearest)
  return accuracy,classification_report_result_k_nearest,predict_for_target_k_nearest

## LDA Run on 50 50 Split

In [ ]:
training_data = pd.DataFrame(odd_data_matrix)
testing_data=pd.DataFrame(even_data_matrix)
number_of_class_samples=np.full((40), 5, dtype=int)
lda_projected_training_set,lda_projected_testing_set=LDA(training_data,testing_data,number_of_class_samples=number_of_class_samples)

training_labels=training_data.iloc[:,0]
training_labels=training_labels.astype('int')
testing_labels=testing_data.iloc[:,0]
testing_labels=testing_labels.astype('int')

_,classification_report_knn,_=KNN(1,training_labels,testing_labels,lda_projected_training_set,lda_projected_testing_set)
print(classification_report_knn)

#LDA Classifier Tuning

In [ ]:
parameters=np.array([1,3,5,7])
lda_accuracies=np.empty((4))
training_labels=training_data.iloc[:,0]
training_labels=training_labels.astype('int')
testing_labels=testing_data.iloc[:,0]
testing_labels=testing_labels.astype('int')
index=0
for parameter in parameters :
  lda_accuracies[index],_,_=KNN(parameter,training_labels,testing_labels,lda_projected_training_set,lda_projected_testing_set)
  index+=1

In [ ]:
plt.plot(parameters,lda_accuracies)
plt.show()

#PCA Classifier Tuning

In [ ]:
pca_tuning_accuracies = list()
for i in [0.8,0.85,0.9,0.95]:
  accuracies = pca_at_K(odd_data_matrix,even_data_matrix,i,[1,3,5,7])
  pca_tuning_accuracies.append(accuracies)

In [ ]:
alphas = [0.8,0.85,0.9,0.95]
figure, axis = plt.subplots(2, 2)
axis[0, 0].plot(alphas,pca_tuning_accuracies[0] )
axis[0, 0].set_title("K=1")

axis[0, 1].plot(alphas, pca_tuning_accuracies[1])
axis[0, 1].set_title("K=3")
  
axis[1, 0].plot(alphas,  pca_tuning_accuracies[2])
axis[1, 0].set_title("K=5")
  
axis[1, 1].plot(alphas,  pca_tuning_accuracies[3])
axis[1, 1].set_title("K=7")
plt.subplots_adjust(hspace=0.8,wspace = 0.8)
print(pca_tuning_accuracies[1])
print(pca_tuning_accuracies[0])
print(pca_tuning_accuracies[2])
plt.show()

# (70-30) Data split comparison

## LDA

In [ ]:
training_data = pd.DataFrame(seventy_data_matrix)
testing_data=pd.DataFrame(thirty_data_matrix)
number_of_class_samples=np.full((40), 7, dtype=int)
lda_projected_training_set,lda_projected_testing_set=LDA(training_data,testing_data,number_of_class_samples=number_of_class_samples)

training_labels=training_data.iloc[:,0]
training_labels=training_labels.astype('int')
testing_labels=testing_data.iloc[:,0]
testing_labels=testing_labels.astype('int')

_,classification_report_knn,_=KNN(1,training_labels,testing_labels,lda_projected_training_set,lda_projected_testing_set)
isNan()
print(classification_report_knn)

## PCA

In [ ]:
_acc,classification_report_knn,_2 = pca(seventy_data_matrix,thirty_data_matrix,[0.8,0.85,0.9,0.95],1)
# for i in classification_report_knn:
#   print(i)
for i in _acc:
  print(i)

#Compare vs Non-Faces

## Generate Faces Vs Non-Faces Dataset

In [ ]:

import numpy as np
import PIL
an_image = PIL.Image.open("/content/drive/MyDrive/prassignment1/s1/1.pgm")
image_sequence = an_image.getdata()
image_array = np.array(image_sequence)
data_matrix = np.full([10305,1],None)
paths=list()
for i in range(40):
  num_of_file=str(i+1)
  path="/content/drive/MyDrive/prassignment1/s{}".format(num_of_file)
  for j in range(10):
    num_of_image=str(j+1)
    full_path=path+"/"+num_of_image+".pgm"
    an_image = PIL.Image.open(full_path)
    image_sequence = an_image.getdata()
    image_array = np.array(image_sequence)
    image_array_labeled = np.insert(image_array,0,0,axis=0)
    data_matrix=np.c_[data_matrix,image_array_labeled]
    paths.append(full_path)

data_matrix=data_matrix[:,1:]
data_matrix=np.transpose(data_matrix)
for i in range(800): 
  path="/content/drive/MyDrive/nonfaces/ ({}).pgm".format(i+1)
  an_image = PIL.Image.open(path)
  image_sequence = an_image.getdata()
  image_array = np.array(image_sequence)
  image_array_labeled = np.insert(image_array,0,1,axis=0)
  data_matrix=np.append(data_matrix,[image_array_labeled],axis=0)
  paths.append(path)
even_data_matrix=np.full([1,10305],None)
odd_data_matrix=np.full([1,10305],None)
for i in range(400):
  if i%2==0:
    even_data_matrix=np.append(even_data_matrix,[data_matrix[i]],axis=0)
    even_data_matrix=np.append(even_data_matrix,[data_matrix[i+400]],axis=0)
  else:
    odd_data_matrix=np.append(odd_data_matrix,[data_matrix[i]],axis=0)
    odd_data_matrix=np.append(odd_data_matrix,[data_matrix[i+400]],axis=0)

even_data_matrix=np.delete(even_data_matrix,0,0)
odd_data_matrix=np.delete(odd_data_matrix,0,0)
print(even_data_matrix)
print(odd_data_matrix)
print(even_data_matrix.shape)
print(odd_data_matrix.shape)




##Compute the Accuracy Using A Specific Number of Eigenvectors

In [ ]:
def compute_accuracy(projection_matrix,training_data,testing_data,number_of_class_samples=None):
 
  
  lda_projected_training_set,lda_projected_testing_set=LDA(training_data,testing_data,projection_matrix=projection_matrix,number_of_class_samples=number_of_class_samples)
  training_labels=training_data.iloc[:,0]
  training_labels=training_labels.astype('int')
  testing_labels=testing_data.iloc[:,0]
  testing_labels=testing_labels.astype('int')
  return KNN(1,training_labels,testing_labels,lda_projected_training_set,lda_projected_testing_set)

In [ ]:
def compute_best_projection_matrix(comp_training_data,comp_testing_data,comp_lda_eigen_values,comp_lda_eigen_vectors,number_of_features=10304):
  highestAccuracy=0;
  noEigenVectors=0;
  comp_best_projection_matrix=None
  for i in range(10,101,10):
    comp_lda_projection_matrix=compute_projection_matrix(eigen_values=comp_lda_eigen_values,eigen_vectors=comp_lda_eigen_vectors,number_of_features=10304,number_of_dominant_eigen_vectors=i)
    accuracy,_,_=compute_accuracy(comp_lda_projection_matrix,comp_training_data,comp_testing_data)
    print("Current Accuracy ",accuracy," With ",i," Eigen Vectors")
    if(accuracy>highestAccuracy):
      highestAccuracy=accuracy;
      noEigenVectors=i
      comp_best_projection_matrix=comp_lda_projection_matrix
    
  print("The Highest Accuracy Is ",highestAccuracy,"With ",noEigenVectors," Eigen Vectors")
  return comp_best_projection_matrix

In [ ]:
even_paths=list()
odd_paths=list()

In [ ]:
##swicth 1 -> 100 non face
##switch 2 -> 200 non face
## can be used till switch 8
def prepare_data(data_matrix,switch):
 limit=switch*100
 even_data_matrix=np.full([1,10305],None)
 odd_data_matrix=np.full([1,10305],None)
 for i in range(400):
    if i<limit:
      if i%2==0:
          even_data_matrix=np.append(even_data_matrix,[data_matrix[i]],axis=0)
          even_paths.append(paths[i])
          even_data_matrix=np.append(even_data_matrix,[data_matrix[i+400]],axis=0)
          even_paths.append(paths[i+400])
      else:
          odd_data_matrix=np.append(odd_data_matrix,[data_matrix[i]],axis=0)
          odd_paths.append(paths[i])
          odd_data_matrix=np.append(odd_data_matrix,[data_matrix[i+400]],axis=0)
          odd_paths.append(paths[i+400])
    else:
      if i%2==0:
          even_data_matrix=np.append(even_data_matrix,[data_matrix[i]],axis=0)
          even_paths.append(paths[i])
      else:
          odd_data_matrix=np.append(odd_data_matrix,[data_matrix[i]],axis=0)
 if limit==500:
    for i in range(100):
      if i%2==0:
        even_data_matrix=np.append(even_data_matrix,[data_matrix[i+800]],axis=0)
        even_paths.append(paths[i+800])
      else:
        odd_data_matrix=np.append(odd_data_matrix,[data_matrix[i+800]],axis=0)
 if limit == 600:
      for i in range(200):
        if i%2==0:
          even_data_matrix=np.append(even_data_matrix,[data_matrix[i+800]],axis=0)
          even_paths.append(paths[i+800])
        else:
          odd_data_matrix=np.append(odd_data_matrix,[data_matrix[i+800]],axis=0)
 if limit == 700:
      for i in range(300):
        if i%2==0:
          even_data_matrix=np.append(even_data_matrix,[data_matrix[i+800]],axis=0)
          even_paths.append(paths[i+800])
        else:
          odd_data_matrix=np.append(odd_data_matrix,[data_matrix[i+800]],axis=0)           
 if limit == 800:
      for i in range(400):
        if i%2==0:
          even_data_matrix=np.append(even_data_matrix,[data_matrix[i+800]],axis=0)
          even_paths.append(paths[i+800])
        else:
          odd_data_matrix=np.append(odd_data_matrix,[data_matrix[i+800]],axis=0)   
 even_data_matrix=np.delete(even_data_matrix,0,0)
 odd_data_matrix=np.delete(odd_data_matrix,0,0)            
 return even_data_matrix,odd_data_matrix

In [ ]:
failure_paths=list()
accuracies=np.empty((8))
classification_reports=list()
predicted_labels=list()
number_of_class_samples=np.empty((2),dtype=int)
number_of_class_samples[0]=200
for i in range(1,9,1):
  even_data_matrix,odd_data_matrix=prepare_data(data_matrix,i)
  failure_paths.append(even_paths)
  even_paths=list()
  comp_training_data=pd.DataFrame(odd_data_matrix)
  comp_testing_data=pd.DataFrame(even_data_matrix)

  comp_training_labels=comp_training_data.iloc[:,0]
  comp_training_labels=comp_training_labels.astype('int')
  comp_testing_labels=comp_testing_data.iloc[:,0]
  comp_testing_labels=comp_testing_labels.astype('int')
  number_of_class_samples[1]=(i*100)/2

  
  lda_projected_training_set,lda_projected_testing_set=LDA(comp_training_data,comp_testing_data,number_of_classes=2,number_of_features=10304,number_of_class_samples=number_of_class_samples,number_of_dominant_eigen_vectors=1)
  accuracies[i-1],_1,_2=KNN(1,comp_training_labels,comp_testing_labels,lda_projected_training_set,lda_projected_testing_set)
  #predicted_labels.append(_2)
  classification_reports.append(_1)
  confusion_matrix_knn = confusion_matrix(comp_testing_labels,_2)
  disp=ConfusionMatrixDisplay(confusion_matrix=confusion_matrix_knn,display_labels=['Faces','NonFaces'])
  disp.plot()
  plt.show()
  gc.collect()

In [ ]:
number_of_nonfaces=[100,200,300,400,500,600,700,800]
plt.plot(number_of_nonfaces,accuracies)
plt.show()

In [ ]:
pca_accuracies=list()
pca_classification_reports=list()
pca_predicted_labels=list()
pca_conf_matrices=list()
for i in range(1,9,1):
  even_data_matrix,odd_data_matrix=prepare_data(data_matrix,i)
  nonfaces_pca_acc,_class_report,_pred_labels,conf_matrix=pca(odd_data_matrix,even_data_matrix,[0.8,0.85,0.9,0.95],1)
  pca_predicted_labels.append(_pred_labels)
  pca_classification_reports.append(_class_report)
  pca_conf_matrices.append(conf_matrix)
  pca_accuracies.append(nonfaces_pca_acc)

##Plotting the Accuracy vs number of nonfaces while fixing the number of faces

In [ ]:
number_of_nonfaces=[100,200,300,400,500,600,700,800]
# plt.plot(number_of_nonfaces,pca_accuracies)
# plt.show()
pca_acc_by_alpha = list()
for i in range(4):
  current_acc = list()
  for j in range(8):
    current_acc.append(pca_accuracies[j][i])
  pca_acc_by_alpha.append(current_acc)
  plt.plot(number_of_nonfaces,current_acc)
  plt.show()
# print(pca_acc_by_alpha)
# print(pca_accuracies)

In [ ]:
for i in range(8):
  for j in range(4):
    disp=ConfusionMatrixDisplay(confusion_matrix=pca_conf_matrices[i][j],display_labels=['Faces','NonFaces'])
    disp.plot()
    plt.show()

##Show Success and Failure Cases

### LDA

In [ ]:
failure_casses=list()
even_data_matrix,odd_data_matrix=prepare_data(data_matrix,1)
comp_training_data=pd.DataFrame(odd_data_matrix)
comp_testing_data=pd.DataFrame(even_data_matrix)
comp_testing_labels=comp_testing_data.iloc[:,0]
comp_testing_labels=comp_testing_labels.astype('int')
for i in range(5):
  failure_casses.append(np.invert(np.in1d(predicted_labels[i],comp_testing_labels)))
for i in range(5):
  print('passes of number of non faces {}'.format((i+1)*100))
  failure_paths_i=failure_paths[i]
  for x in failure_paths_i[failure_casses[i]]:
    print(x)  